In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys

os.chdir('/content/drive/MyDrive/Colab_Notebooks/Analyse_Avis')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/Analyse_Avis')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Modèle pour prédire le score ou la note que le client donnerai au service rendu sur la base de son commentaire

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.utils import shuffle
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
import zipfile

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

#Importation du modèle BERTDISTIL (variante très léger de BERT) sur Hugging Face
from transformers import (
    DistilBertTokenizer,#Tokenizer pour DistilBert
    TFDistilBertModel,#Model DistilBert
    DataCollatorWithPadding,#Gestion du padding pour avoir les sequence de même taille
    BertTokenizer,#Model Bert
    TFBertModel#Model Bert
)

#Outils pour le suivi et la visualisation :
from tqdm.notebook import tqdm #Pour afficher barre de progression dans jupyter notebook
import itertools #Pour gérer les itérations
import multiprocessing #Pour faire de la parallélisation
import os

np.random.seed(987654321)
tf.random.set_seed(987654321)

In [3]:
# Chemin vers le fichier ZIP
zip_file_path = "archive.zip"

# Ouvrir l'archive ZIP
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # Lister les fichiers dans l'archive
    print("Fichiers dans l'archive ZIP :", z.namelist())

    # Spécifier le nom du fichier CSV à charger
    csv_file_name = z.namelist()[0]  # Remplacer par le nom exact si connu

    # Charger le fichier CSV dans un DataFrame pandas
    with z.open(csv_file_name) as f:
        data = pd.read_csv(f)

# Afficher les premières lignes du DataFrame
data.head()

Fichiers dans l'archive ZIP : ['amazon_reviews.csv']


,userName,verified,itemName,description,image,brand,feature,category,price,rating,reviewTime,summary,reviewText,vote
0,Amazon Customer,True,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",['https://images-na.ssl-images-amazon.com/imag...,Toblerone,[],Prime_Pantry,$1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0
1,Carla,True,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",['https://images-na.ssl-images-amazon.com/imag...,Prince Of Peace,[],Grocery_and_Gourmet_Food,$6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0
2,Jaclyn,True,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",['https://images-na.ssl-images-amazon.com/imag...,Pet Champion,"['Features Bright Pink Lemonade pattern', 'Dua...",Pet_Supplies,$7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0
3,Christinne M,False,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,['https://images-na.ssl-images-amazon.com/imag...,KOH-I-NOOR,"['Rich, pigmented, solid color encased in lacq...",Arts_Crafts_and_Sewing,$14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0
4,Sandy,True,"Parker Urban Premium Ebony Metal Chiseled, Bal...","[""Stunning craftsmanship based on a classic st...",['https://images-na.ssl-images-amazon.com/imag...,Parker,['Ebony black with metallic highlights and a g...,Office_Products,NaN,5.0,2018-01-01,I love this pen,"I love this pen! I love the shape of it, the f...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551159 entries, 0 to 551158
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userName     551026 non-null  object 
 1   verified     551159 non-null  bool   
 2   itemName     551158 non-null  object 
 3   description  551151 non-null  object 
 4   image        551159 non-null  object 
 5   brand        547287 non-null  object 
 6   feature      551159 non-null  object 
 7   category     551159 non-null  object 
 8   price        476825 non-null  object 
 9   rating       551159 non-null  float64
 10  reviewTime   551159 non-null  object 
 11  summary      550916 non-null  object 
 12  reviewText   550731 non-null  object 
 13  vote         551159 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 55.2+ MB


In [5]:
# Sélectionner uniquement les colonnes 'rating' et 'reviewText' avec .loc
data = data.loc[:, ['rating', 'reviewText']]

# Convertir la colonne 'rating' en type catégorie
data['rating'] = data['rating'].astype('int')

# Afficher les informations du DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551159 entries, 0 to 551158
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   rating      551159 non-null  int64 
 1   reviewText  550731 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.4+ MB


## Stat Descriptive

In [6]:
data.loc[:, 'rating'].value_counts()

,count
rating,
5,379783
4,73303
3,40639
1,34021
2,23413


In [7]:
data.loc[:, 'reviewText'][10]

'Good for the price, ships quickly.'

## Gestion des NA

In [8]:
data.isnull().mean()*100

,0
rating,0.000000
reviewText,0.077655


In [9]:
data = data.dropna()
data.isnull().mean()*100

,0
rating,0.0
reviewText,0.0


## Prétraitement pour le NLP et le fine tuning

### Paramètre

In [10]:
vocab_size = 30000 #Taille max du dictionnaire pour la tokenisation ici limité aux 30 000 mots les plus fréquents dans les données d'entraînement
hide_most_frequently = 0 #ignoré les mots les plus fréquent comme "le, la , ..." qui sont peu informatif dans certaines contexte
#ici 0 nous permet de rien ignorer
review_len = 512 #fixe la longueur maximale des séquences de texte traitées ici 512 tokens, la limite standard pour BERT.

epochs = 10
batch_size = 100

fit_verbosity = 1
scale = 1

# Définir le nombre maximal de cœurs disponibles
n_cpus = os.cpu_count()

## Chargement du model BERT préentrainé et son tokenizer

In [11]:
#Fonction de chargement de modèle BERT
def load_model(distil):
  if distil:
      bert_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
      tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
  else:
      bert_model = TFBertModel.from_pretrained("bert-base-uncased")
      tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
  return bert_model, tokenizer

# Indiquer si vous voulez utiliser DistilBERT (True) ou BERT classique (False)
distil = True  # Remplacez par False si vous voulez utiliser BERT classique

bert_model, tokenizer = load_model(distil)
bert_model.summary()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

Model: "tf_distil_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
Total params: 66362880 (253.15 MB)
Trainable params: 66362880 (253.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Tokenisation

In [12]:
#Cette fonction prend un sample (ici, une critique sous forme de texte) puis le convertit en tokens adaptés au modèle BERT.
def tokenize_sample(sample):
    return tokenizer(sample, truncation=True, max_length=review_len)

#Tokeniser un ensemble de données (dataset) en utilisant tous les cœurs CPU disponibles pour accélérer le processus.
def distributed_tokenize_dataset(dataset):
    dataset = list(dataset)
    #Pour faire des calcul en parallèle pour la tokenisation sur plusieurs coeurs
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        tokenized_ds = list(tqdm(
            pool.imap(tokenize_sample, dataset),  # Tokenisation des textes bruts
            total=len(dataset)
        ))
    return tokenized_ds

X = data.loc[:, 'reviewText']
y = data.loc[:, 'rating']

#Séparation en base de test et d'entrainement
text_x_train,text_x_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#convertit les étiquettes en tableau NumPy de type float32, pour assurer la compatibilité avec les couches et fonctions TensorFlow/Keras, qui attendent souvent des données de type flottant.
y_train = np.asarray(y_train).astype('float32')
y_test  = np.asarray(y_test ).astype('float32')


tokenized_x_train = distributed_tokenize_dataset(text_x_train)
tokenized_x_test = distributed_tokenize_dataset(text_x_test)

  0%|          | 0/440584 [00:00<?, ?it/s]

  0%|          | 0/110147 [00:00<?, ?it/s]

In [13]:
#S'assure que toutes les séquences ont la même longueur en les remplissant (padding) à la longueur maximale.
data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")
#Par exemple
#data_collator(tokenized_x_train)

# transforme les données tokenisées en un format de dataset TensorFlow prêt pour l’entraînement du modèle
def make_dataset(x, y):
    collated = data_collator(x)#Faire le padding sur toutes les reviews

    """
    Chaque échantillon dans dataset est une tuple de trois éléments :
    - input_ids (séquence tokenisée),
    - attention_mask (indiquant les tokens réels vs padding) avec attention_mask un masque indiquant quelles positions correspondent aux tokens réels et lesquelles sont des tokens de padding (utile pour que le modèle ignore le padding). ,
    - y (label correspondant).
    """
    dataset = tf.data.Dataset.from_tensor_slices(
        (collated['input_ids'], collated['attention_mask'], y)
    )

    #Transformation du dataset en format adapté pour BERT qui attendent en entrée à la fois input_ids et attention_mask ensemble.
    """
    - Transforme chaque échantillon (input_ids, attention_mask, y) en ((input_ids, attention_mask), y).
    - shuffle(25000) : Mélange les données pour réduire la corrélation entre les échantillons consécutifs (valeur 25000 pour un mélange complet si c'est la taille du dataset).
    - batch(batch_size) : Regroupe les données en lots de taille batch_size (défini ailleurs comme 32).
    """
    transformed_dataset = (
        dataset.map(
            lambda x, y, z: ((x, y), z)
        ).shuffle(25000).batch(batch_size)
    )

    return transformed_dataset

train_ds = make_dataset(tokenized_x_train, y_train)
test_ds = make_dataset(tokenized_x_test, y_test)

for x, y in train_ds:
    print(x)
    print(y)
    break

(<tf.Tensor: shape=(100, 512), dtype=int32, numpy=
array([[ 101, 2573, 2307, ...,    0,    0,    0],
       [ 101, 2307, 2012, ...,    0,    0,    0],
       [ 101, 7929, 2422, ...,    0,    0,    0],
       ...,
       [ 101, 6919, 3280, ...,    0,    0,    0],
       [ 101, 2293, 1996, ...,    0,    0,    0],
       [ 101, 1996, 4937, ...,    0,    0,    0]], dtype=int32)>, <tf.Tensor: shape=(100, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>)
tf.Tensor(
[4. 4. 3. 5. 5. 5. 5. 4. 5. 5. 5. 5. 1. 5. 5. 2. 1. 4. 5. 4. 1. 3. 5. 5.
 5. 2. 5. 5. 5. 5. 5. 5. 5. 1. 3. 5. 5. 5. 4. 2. 5. 4. 5. 5. 3. 5. 5. 5.
 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 3. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 1.
 5. 5. 5. 5. 5. 5. 5. 5. 1. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 5. 3. 1. 5.
 5. 5. 4. 4.], shape=(100,), dtype=float32)


## Ajout de la tete de regression au modèle BERT


In [14]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

class RegressionModel(Model):
    def __init__(self, bert_model):
        super(RegressionModel, self).__init__()
        self.bert_model = bert_model
        """
        - pre_classifier : Une couche dense avec 1000 unités et une activation ReLU pour affiner la représentation extraite de BERT.
        - dropout : Couche Dropout avec un taux de 0.1 pour éviter le surapprentissage.
        - regressor : Couche dense finale avec 1 unité pour prédire une valeur réelle.
        """
        self.pre_classifier = Dense(1000, activation='relu')
        self.dropout = Dropout(0.1)
        self.regressor = Dense(1)  # Une seule unité pour la régression

    def call(self, x):
        """
        Applique BERT et les couches pour produire une prédiction réelle.
        """
        # Sortie de BERT
        x = self.bert_model(x)
        x = x.last_hidden_state

        # Extraire le token [CLS]
        x = x[:, 0]

        # Pré-classificateur
        x = self.pre_classifier(x)

        # Dropout
        x = self.dropout(x)

        # Prédiction réelle
        x = self.regressor(x)
        return x

    def predict(self, texts, tokenizer, data_collator):
        """
        Prédit une valeur réelle pour une liste de phrases.

        Args:
        texts (list): Liste de phrases à analyser.
        tokenizer: Tokenizer utilisé pour les textes.
        data_collator: Fonction pour structurer les données après tokenisation.

        Returns:
        list: Liste de prédictions réelles.
        """
        # Tokenisation des textes
        tokenized_texts = [tokenizer(text) for text in texts]
        collated = data_collator(tokenized_texts)

        # Convertir les données en tenseurs TensorFlow
        inputs = {
            'input_ids': tf.convert_to_tensor(collated['input_ids']),
            'attention_mask': tf.convert_to_tensor(collated['attention_mask'])
        }

        # Effectuer les prédictions
        predictions = self(inputs)

        return predictions.numpy().flatten().tolist()


In [15]:
#Création du modèle
model = RegressionModel(bert_model)

#Test
x = next(iter(train_ds))[0]
model(x)
model.summary()

Model: "regression_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (100, 1000)                 │         769,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (100, 1000)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (100, 1)                    │           1,001 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 770,001 (2.94 MB)

 Trainable params: 770,001 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

### Entrainement

In [16]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)


history = model.fit(
    train_ds,  # Dataset d'entraînement
    validation_data=test_ds,  # Dataset de validation
    epochs=epochs,
    batch_size=batch_size,
    verbose=fit_verbosity
)


Epoch 1/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 548s 122ms/step - loss: 2.4426 - mean_absolute_error: 1.0881 - val_loss: 0.7266 - val_mean_absolute_error: 0.6098
Epoch 2/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 529s 120ms/step - loss: 0.7030 - mean_absolute_error: 0.5935 - val_loss: 0.6669 - val_mean_absolute_error: 0.5582
Epoch 3/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 527s 120ms/step - loss: 0.6556 - mean_absolute_error: 0.5557 - val_loss: 0.6411 - val_mean_absolute_error: 0.5477
Epoch 4/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 527s 120ms/step - loss: 0.6334 - mean_absolute_error: 0.5384 - val_loss: 0.6281 - val_mean_absolute_error: 0.5286
Epoch 5/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 530s 120ms/step - loss: 0.6191 - mean_absolute_error: 0.5282 - val_loss: 0.6181 - val_mean_absolute_error: 0.5250
Epoch 6/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 526s 119ms/step - loss: 0.6101 - mean_absolute_error: 0.5218 - val_loss: 0.6099 - val_mean_absolute_error: 0.5205
Epoch 7/10
4406/4406 ━━━━━━━━━━━━━━━━━━━━ 526s 119ms/step - loss

In [ ]:
history_data = history.history
# Extraire les pertes pour l'entraînement et la validation
train_loss = history_data['loss']
val_loss = history_data['val_loss']
epochs = range(1, len(train_loss) + 1)

# Tracer le graphique
plt.figure(figsize=(8, 6))
plt.plot(epochs, train_loss, 'o-', label='Training Loss (MSE)', color='blue')
plt.plot(epochs, val_loss, 'o-', label='Validation Loss (MSE)', color='red')
plt.title('Training and Validation Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)
plt.show()

KeyError: 'val_accuracy'

In [ ]:
_, score = model.evaluate(test_ds)
colors = sns.color_palette('pastel')[2:]
accuracy_score = [score, 1 - score]
plt.pie(
    accuracy_score,
    labels=["Accurate", "Mistaken"],
    colors=colors,
    autopct=lambda val: f"{val:.2f}%",
    explode=(0.0, 0.1)
)
plt.show()

### Test sur des exemples

In [1]:
# Exemples de phrases à tester
examples = [
    "This movie was absolutely fantastic! The acting was superb and the story was captivating.",
    "I didn't like this movie at all. It was boring and way too long.",
    "The plot was interesting, but the characters lacked depth.",
    "An amazing experience!.",
    "Terrible Toys.",
    "J'aime pas trop le produit"
]

# Utilisation de la nouvelle méthode `predict`
results = model.predict(examples)

# Affichage des résultats
for example, result in zip(examples, results):
    print(f"Phrase: {example}\nPrédiction: {result}\n")

NameError: name 'model' is not defined